# Split up in letters

In this notebook we'll combine all the ocr-ed pages into one large text. This text will be split into individual letters using the seperator *****.

We'll store the individual letters in a database table `letters`. It'll have the following columns:
- `id`: unique identifier
- `letter`: the text of the letter
- `start_page`: the page number where the letter starts
- `end_page`: the page number where the letter ends

In [23]:

import re
from db import Database

db = Database()

In [24]:
db.execute("""
    CREATE TABLE IF NOT EXISTS letters (
        id SERIAL PRIMARY KEY,
        raw TEXT,
        start_page INTEGER,
        end_page INTEGER,
        markdown TEXT,
        english TEXT,
        sender TEXT,
        recipient TEXT,
        subject TEXT,
        send_date DATE,
        location TEXT,
        geolocation FLOAT[],
        language TEXT,
        summary TEXT
    )
""")

In [25]:
# Fetch all pages with content and concatenate them into a single string, separated by newlines
db.execute("""
    SELECT ocr_improved, page_number 
    FROM pages 
    WHERE ocr_improved IS NOT NULL AND page_number > 22
    ORDER BY page_number
    """)

pages = db.fetchall()
print(f"Found {len(pages)} pages")

book = "\n".join((page[0] if page[0] is not None else "") for page in pages)

# Split the book into letters based on 3 to 7 consecutive asterisks
letters = re.split(r"\*{3,7}", book)

print(f"Found {len(letters)} letters")

for i, letter in enumerate(letters):
    # Skip empty letters, or letters that only contain whitespace or new lines
    if not letter.strip():
        continue
    
    
    start_snippet, end_snippet = letter[:50], letter[-50:]
    print(f"First Lines: {letter[:10]}")

    # Check if the letter already exists in the database
    db.execute("""SELECT id FROM letters WHERE raw = %s""", (letter,))
    existing_letter = db.fetchone()

    if existing_letter:
        print(f"Letter {i + 1} already exists in the database.")
        continue

    # Find the start and end page numbers based on these snippets
    db.execute("""
        SELECT min(page_number), max(page_number)
        FROM pages
        WHERE ocr_improved LIKE %s OR ocr_improved LIKE %s
    """, (f"%{start_snippet}%", f"%{end_snippet}%"))

    start_page, end_page = db.fetchone()

    # Insert the letter with its start and end page numbers into the database
    if start_page and end_page:
        db.execute("INSERT INTO letters (raw, start_page, end_page) VALUES (%s, %s, %s)", (letter, start_page, end_page))
        print(f"Inserted letter {i + 1} of {len(letters)}")
    else:
        print(f"Page range for letter {i + 1} could not be determined.")


Found 11 pages
Found 5 letters
First Lines: BRIEVEN VA
Inserted letter 1 of 5
First Lines: 
VONDEL AA
Inserted letter 2 of 5
First Lines: 
VONDEL AA
Inserted letter 3 of 5
First Lines: 

WILLEM V
Inserted letter 4 of 5
First Lines: 

WILLEM V
Inserted letter 5 of 5
